In [1]:
import json
import nltk

### 将文本加载为json格式

In [2]:
input_file = 'tweets.txt'
with open(input_file, 'r') as f:
    lines = f.readlines()
items = [json.loads(x) for x in lines]

### 定义预处理类
* 大写转小写，去除尾部网址
* 分词
* 去除标点符号和停用词

In [3]:
class Preprocess:
    def __init__(self):
        self.punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%', '...']
        self.stop_words = set(nltk.corpus.stopwords.words('english'))
    def __call__(self,query):
        query = query.lower().split('http')[0]
        query = nltk.word_tokenize(query)
        query = [x for x in query if x not in self.punctuations and x not in self.stop_words]
        return query

In [4]:
preprocess = Preprocess()
tweets = [preprocess(x['text']) for x in items]

### 统计词频

In [5]:
dictionary = {}
for i, tweet in enumerate(tweets):
    freq_dist = nltk.FreqDist(tweet)
    for word, freq in freq_dist.items():
        if word in dictionary:
            dictionary[word].add(i)
        else:
            dictionary[word] = set([i])

In [6]:
class Query_process(Preprocess):
    def __init__(self):
        super().__init__()
    def __call__(self,query):
        print('原始查询语句: {}'.format(query))
        query = query.lower()
        query = nltk.word_tokenize(query)
        query = ' '.join([x for x in query if x not in self.punctuations and x not in self.stop_words or x in ['not','or','|','-']])
        query = query.replace(' or ', '|').replace(' | ','|').replace(' not ', '-').replace(' - ','-').replace(' ', '&')
        query = query.replace('|', ' | ').replace('&',' & ').replace('-',' - ')
        print('转化为表达式: {}\n'.format(query))
        query = ' '.join(['dictionary[\''+x+'\']' if x != '&' and x != '|' and x != '-' else x for x in query.split()])
        try:
            result = eval(query)
            return eval(query)
        except:
            return None

    def get_query_result_text(self,query):
        item_list = self.__call__(query)
        if item_list is None:
            return None
        result = [items[i]['text'] for i in item_list]
        result = '\n\n'.join(result)
        return result

# 显示查询交互

In [7]:
import tkinter as tk
from IPython.display import clear_output

In [8]:
win = tk.Tk()
win.title('查询')
win.geometry('700x100')

query = tk.StringVar()
entry = tk.Entry(win, relief='sunken',font='Calibri 23',width=33,textvariable=query)
entry.place(relx=0.43,rely=0.5,anchor='center')

def button_command():
    clear_output()
    query_sentence = query.get()
    print(query_process.get_query_result_text(query_sentence))

button = tk.Button(win, text='查询', height=2,width=8,command=button_command)
button.place(relx=0.9,rely=0.5,anchor='center')

In [9]:
query_process=Query_process()

In [ ]:
win.mainloop()

原始查询语句: Ron Weasley or Birthday not Harry
转化为表达式: ron & weasley | birthday - harry

@MuggleNet #RonWeasleyBirthday Happy Birthday Ron!! "Why can't it be follow the butterflies!" x

Happy birthday to Ronald Bilius Weasley #WeasleyIsOurKing #HPForLife 👍

In honor of Mr. Roger's birthday (he would have been 85 today) I want you to know that YOU'RE SPECIAL.

RT @Tate: Happy birthday Tate Britain, opened on this day 1897! What's been your favourite Tate Britain moment? http://t.co/VPYvtfyUOF

Happy birthday to Big Bird *and* to Mr. Rogers! - Sesame Street http://t.co/Nx7hsTBfHC

Happy Birthday Mr. Rogers: 35 Facts about Mr. Rogers and Why He Was Amazing http://t.co/2LbngSHxVs

Happy Good Neighbor Day! (that's how I'm campaigning for Mr Roger's birthday to be a national holiday) #happybirthday

Why does everyone care about Justin Bieber's birthday? ITS RON WEASLEYS. #potterhead

Photo: sesamestreet: Happy birthday to Big Bird *and* to Mr. Rogers! A true meeting of the minds. http://t.co/feuc